In [28]:
from calendar import c
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mlr import MLR


In [29]:
def calculateEvalMeasures(pred, targets, verbose = True):
    m = len(pred)
    mse = (((pred - targets)**2).sum() / (m))[0]
    mae = ((abs(pred - targets)).sum() / (m))[0]
    rmse = np.sqrt(mse)
    rss = (((targets-pred)**2).sum())[0]
    tss = (((targets-targets.mean())**2).sum())[0]
    r2 = 1 - (rss/tss)
    if verbose:
        print(r2.shape)
        print("== Results ==")
        print("R^2:", r2)
        print("Mean Absolute Error:", mae)
        print("MSE:", mse)
        print("RMSE:", rmse)
    return r2, mae, mse, rmse

# TODO Implement Cross-validation

In [30]:
dataset = pd.read_csv("data/Folds5x2_pp.csv")
mlr = MLR(dataset, "PE")
pred, params = mlr.learnParams(0.1)
# Training evaluation
targets = mlr.target
r2, mae, mse, rmse = calculateEvalMeasures(pred, targets, True)

()
== Results ==
R^2: 0.9283720682372759
Mean Absolute Error: 3.638194702772992
MSE: 20.861769423767388
RMSE: 4.567468601289711
